# Reference  model
The first reference will be a model without any usage of knowledge.
The probability to end up in the class for a test customer is assumed to be *n(customers per class)/all(customers)*.

In [7]:
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns
import sys

%matplotlib inline
from matplotlib import pyplot as plt

In [8]:
sns.set_style('ticks')

In [9]:
sys.path.append("/home/mschlupp/pythonTools")
tmp = %pwd
files_dir = tmp + "/files/" 

In [11]:
train = pd.read_csv(files_dir+'gender_age_train.csv')

In [29]:
groups = train.groupby('group').count()

In [30]:
groups.device_id = groups.device_id/len(train.age)

In [31]:
groups.device_id

group
F23-      0.067654
F24-26    0.056132
F27-28    0.041771
F29-32    0.062000
F33-42    0.074499
F43+      0.056186
M22-      0.100315
M23-26    0.128676
M27-28    0.072945
M29-31    0.097917
M32-38    0.126948
M39+      0.114957
Name: device_id, dtype: float64